<a href="https://colab.research.google.com/github/ngocbaosp/ML-Projects/blob/master/Content_Based_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np

In [0]:
movies = pd.read_csv('testdata.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')
credits.columns = ['id','tittle','cast','crew']

In [0]:
from ast import literal_eval
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
  
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

def convertTime(minutes): 
    hour = minutes // 60
    minutes = minutes % 60   
    return "%dh%02d" % (hour, minutes) 
  
def paserData(input):
  
  data = input.copy()
  
  features = ['cast', 'crew', 'keywords', 'genres','production_companies']
  for feature in features:
    data[feature] = data[feature].apply(literal_eval)
    
  data['director'] = data['crew'].apply(get_director)

  features = ['cast', 'keywords', 'genres','production_companies','crew']
  for feature in features:
    data[feature] = data[feature].apply(get_list)
    
        
    
  newData = data  
  return newData

def combineData(credits,movies):
  imageURL='https://image.tmdb.org/t/p/original'
  
  data = pd.merge(movies,credits, how='inner', on=['id'])
  
  
  data['imgURL'] = imageURL + movies['poster_path']
  
  data = paserData(data)
  
  return data
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']



In [0]:
columns=['id','genres','keywords','title','director','cast']


In [125]:
newMoviesData= combineData(credits,movies)
newMoviesData.head()
movieTitle =['Spider-Man 3','The Amazing Spider-Man', 'Spider-Man 2', 'The Amazing Spider-Man 2', 'Spider-Man', 'Jupiter Ascending']
moviesCF = newMoviesData.loc[newMoviesData['title'].isin(movieTitle)]

moviesCF = moviesCF[columns]
moviesCF

,id,genres,keywords,title,director,cast
0,559,"[Fantasy, Action, Adventure]","[dual identity, amnesia, sandstorm]",Spider-Man 3,Sam Raimi,"[Tobey Maguire, Kirsten Dunst, James Franco]"
1,1930,"[Action, Adventure, Fantasy]","[loss of father, vigilante, serum]",The Amazing Spider-Man,Marc Webb,"[Andrew Garfield, Emma Stone, Rhys Ifans]"
2,558,"[Action, Adventure, Fantasy]","[dual identity, love of one's life, pizza boy]",Spider-Man 2,Sam Raimi,"[Tobey Maguire, Kirsten Dunst, James Franco]"
3,102382,"[Action, Adventure, Fantasy]","[obsession, marvel comic, sequel]",The Amazing Spider-Man 2,Marc Webb,"[Andrew Garfield, Emma Stone, Jamie Foxx]"
4,557,"[Fantasy, Action]","[loss of lover, spider, thanksgiving]",Spider-Man,Sam Raimi,"[Tobey Maguire, Willem Dafoe, Kirsten Dunst]"
5,76757,"[Science Fiction, Fantasy, Action]","[jupiter, space, woman director]",Jupiter Ascending,Lilly Wachowski,"[Mila Kunis, Channing Tatum, Sean Bean]"


In [126]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
cbData = moviesCF.copy()

for feature in features:
    cbData[feature] = cbData[feature].apply(clean_data)
    
cbData

,id,genres,keywords,title,director,cast
0,559,"[fantasy, action, adventure]","[dualidentity, amnesia, sandstorm]",Spider-Man 3,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
1,1930,"[action, adventure, fantasy]","[lossoffather, vigilante, serum]",The Amazing Spider-Man,marcwebb,"[andrewgarfield, emmastone, rhysifans]"
2,558,"[action, adventure, fantasy]","[dualidentity, loveofone'slife, pizzaboy]",Spider-Man 2,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
3,102382,"[action, adventure, fantasy]","[obsession, marvelcomic, sequel]",The Amazing Spider-Man 2,marcwebb,"[andrewgarfield, emmastone, jamiefoxx]"
4,557,"[fantasy, action]","[lossoflover, spider, thanksgiving]",Spider-Man,samraimi,"[tobeymaguire, willemdafoe, kirstendunst]"
5,76757,"[sciencefiction, fantasy, action]","[jupiter, space, womandirector]",Jupiter Ascending,lillywachowski,"[milakunis, channingtatum, seanbean]"


In [55]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])#+ ' '.join(x['production_companies'])
  
  
cbData['soup'] = cbData.apply(create_soup, axis=1)

cbData['soup']

0    dualidentity amnesia sandstorm tobeymaguire ki...
1    lossoffather vigilante serum andrewgarfield em...
2    dualidentity loveofone'slife pizzaboy tobeymag...
3    obsession marvelcomic sequel andrewgarfield em...
4    lossoflover spider thanksgiving tobeymaguire w...
5    jupiter space womandirector milakunis channing...
Name: soup, dtype: object

In [59]:
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(cbData['soup'])
count_matrix

<6x36 sparse matrix of type '<class 'numpy.int64'>'
	with 60 stored elements in Compressed Sparse Row format>

In [0]:
maxtrix = count_matrix.toarray()

In [71]:
spareMatrix = pd.DataFrame(maxtrix,columns = count.get_feature_names())
spareMatrix

,action,adventure,amnesia,andrewgarfield,channingtatum,dualidentity,emmastone,fantasy,jamesfranco,jamiefoxx,jupiter,kirstendunst,lillywachowski,lossoffather,lossoflover,loveofone,marcwebb,marvelcomic,milakunis,obsession,pizzaboy,rhysifans,samraimi,sandstorm,sciencefiction,seanbean,sequel,serum,slife,space,spider,thanksgiving,tobeymaguire,vigilante,willemdafoe,womandirector
0,1,1,1,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0
1,1,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0
2,1,1,0,0,0,1,0,1,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0
3,1,1,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,1,0
5,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1


In [127]:
cbData

,id,genres,keywords,title,director,cast
0,559,"[fantasy, action, adventure]","[dualidentity, amnesia, sandstorm]",Spider-Man 3,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
1,1930,"[action, adventure, fantasy]","[lossoffather, vigilante, serum]",The Amazing Spider-Man,marcwebb,"[andrewgarfield, emmastone, rhysifans]"
2,558,"[action, adventure, fantasy]","[dualidentity, loveofone'slife, pizzaboy]",Spider-Man 2,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
3,102382,"[action, adventure, fantasy]","[obsession, marvelcomic, sequel]",The Amazing Spider-Man 2,marcwebb,"[andrewgarfield, emmastone, jamiefoxx]"
4,557,"[fantasy, action]","[lossoflover, spider, thanksgiving]",Spider-Man,samraimi,"[tobeymaguire, willemdafoe, kirstendunst]"
5,76757,"[sciencefiction, fantasy, action]","[jupiter, space, womandirector]",Jupiter Ascending,lillywachowski,"[milakunis, channingtatum, seanbean]"


In [120]:
X = np.sum(maxtrix, axis = 0)
#X[::-1].sort()
df = pd.DataFrame(X)
df.head()

,0
0,6
1,4
2,1
3,2
4,1


In [102]:
cosine_sim1 = cosine_similarity(count_matrix, count_matrix)
pd.DataFrame(cosine_sim1).round(3).astype(float)

,0,1,2,3,4,5
0,1.000,0.300,0.763,0.300,0.527,0.200
1,0.300,1.000,0.286,0.600,0.211,0.200
2,0.763,0.286,1.000,0.286,0.503,0.191
3,0.300,0.600,0.286,1.000,0.211,0.200
4,0.527,0.211,0.503,0.211,1.000,0.211
5,0.200,0.200,0.191,0.200,0.211,1.000


In [128]:
cbData

,id,genres,keywords,title,director,cast
0,559,"[fantasy, action, adventure]","[dualidentity, amnesia, sandstorm]",Spider-Man 3,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
1,1930,"[action, adventure, fantasy]","[lossoffather, vigilante, serum]",The Amazing Spider-Man,marcwebb,"[andrewgarfield, emmastone, rhysifans]"
2,558,"[action, adventure, fantasy]","[dualidentity, loveofone'slife, pizzaboy]",Spider-Man 2,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
3,102382,"[action, adventure, fantasy]","[obsession, marvelcomic, sequel]",The Amazing Spider-Man 2,marcwebb,"[andrewgarfield, emmastone, jamiefoxx]"
4,557,"[fantasy, action]","[lossoflover, spider, thanksgiving]",Spider-Man,samraimi,"[tobeymaguire, willemdafoe, kirstendunst]"
5,76757,"[sciencefiction, fantasy, action]","[jupiter, space, womandirector]",Jupiter Ascending,lillywachowski,"[milakunis, channingtatum, seanbean]"


In [90]:
Threshold = 1
Y = X>Threshold
Y

T = maxtrix[ : , Y]
T.shape
T




array([[1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1],
       [1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]])

In [101]:
cosine_sim2 = cosine_similarity(T, T)
pd.DataFrame(cosine_sim2).round(3).astype(float)

,0,1,2,3,4,5
0,1.000,0.433,1.000,0.433,0.791,0.500
1,0.433,1.000,0.433,1.000,0.365,0.577
2,1.000,0.433,1.000,0.433,0.791,0.500
3,0.433,1.000,0.433,1.000,0.365,0.577
4,0.791,0.365,0.791,0.365,1.000,0.632
5,0.500,0.577,0.500,0.577,0.632,1.000


In [99]:
pd.DataFrame(cosine_sim1).round(3).astype(float)

,0,1,2,3,4,5
0,1.000,0.300,0.763,0.300,0.527,0.200
1,0.300,1.000,0.286,0.600,0.211,0.200
2,0.763,0.286,1.000,0.286,0.503,0.191
3,0.300,0.600,0.286,1.000,0.211,0.200
4,0.527,0.211,0.503,0.211,1.000,0.211
5,0.200,0.200,0.191,0.200,0.211,1.000


In [98]:
pd.DataFrame(cosine_sim2 - cosine_sim1).round(3).astype(float)

,0,1,2,3,4,5
0,0.000,0.133,0.237,0.133,0.264,0.300
1,0.133,0.000,0.147,0.400,0.154,0.377
2,0.237,0.147,-0.000,0.147,0.288,0.309
3,0.133,0.400,0.147,0.000,0.154,0.377
4,0.264,0.154,0.288,0.154,-0.000,0.422
5,0.300,0.377,0.309,0.377,0.422,-0.000


In [0]:
#https://www.mathopenref.com/cosine.html
#https://dataaspirant.com/2015/04/11/five-most-popular-similarity-measures-implementation-in-python/

In [0]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(inputData,title, cosine_sim,topN):
  
    data = inputData.copy()
  
    #Construct a reverse map of indices and movie titles
    indices = pd.Series(data.index, index=data['title']).drop_duplicates()

    # Get the index of the movie that matches the title
    idx = indices[title]
    
    print(idx)

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    print(sim_scores)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:topN]
    
    print(sim_scores)

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]
    result = data.iloc[movie_indices]
    result['sim_score'] = sim_scores
    return result

In [134]:
#cbResult = get_recommendations(cbData,'The Dark Knight Rises',cosine_sim2,3)
cbResult = get_recommendations(cbData,'The Amazing Spider-Man',cosine_sim1,10)
cbResult

1
[(1, 0.9999999999999999), (3, 0.6), (0, 0.30000000000000004), (2, 0.2860387767736777), (4, 0.21081851067789195), (5, 0.2)]
[(3, 0.6), (0, 0.30000000000000004), (2, 0.2860387767736777), (4, 0.21081851067789195), (5, 0.2)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,genres,keywords,title,director,cast,sim_score
3,102382,"[action, adventure, fantasy]","[obsession, marvelcomic, sequel]",The Amazing Spider-Man 2,marcwebb,"[andrewgarfield, emmastone, jamiefoxx]","(3, 0.6)"
0,559,"[fantasy, action, adventure]","[dualidentity, amnesia, sandstorm]",Spider-Man 3,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]","(0, 0.30000000000000004)"
2,558,"[action, adventure, fantasy]","[dualidentity, loveofone'slife, pizzaboy]",Spider-Man 2,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]","(2, 0.2860387767736777)"
4,557,"[fantasy, action]","[lossoflover, spider, thanksgiving]",Spider-Man,samraimi,"[tobeymaguire, willemdafoe, kirstendunst]","(4, 0.21081851067789195)"
5,76757,"[sciencefiction, fantasy, action]","[jupiter, space, womandirector]",Jupiter Ascending,lillywachowski,"[milakunis, channingtatum, seanbean]","(5, 0.2)"


In [132]:
cbData

,id,genres,keywords,title,director,cast
0,559,"[fantasy, action, adventure]","[dualidentity, amnesia, sandstorm]",Spider-Man 3,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
1,1930,"[action, adventure, fantasy]","[lossoffather, vigilante, serum]",The Amazing Spider-Man,marcwebb,"[andrewgarfield, emmastone, rhysifans]"
2,558,"[action, adventure, fantasy]","[dualidentity, loveofone'slife, pizzaboy]",Spider-Man 2,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
3,102382,"[action, adventure, fantasy]","[obsession, marvelcomic, sequel]",The Amazing Spider-Man 2,marcwebb,"[andrewgarfield, emmastone, jamiefoxx]"
4,557,"[fantasy, action]","[lossoflover, spider, thanksgiving]",Spider-Man,samraimi,"[tobeymaguire, willemdafoe, kirstendunst]"
5,76757,"[sciencefiction, fantasy, action]","[jupiter, space, womandirector]",Jupiter Ascending,lillywachowski,"[milakunis, channingtatum, seanbean]"


In [135]:
#cbResult = get_recommendations(cbData,'The Dark Knight Rises',cosine_sim2,3)
cbResult = get_recommendations(cbData,'The Amazing Spider-Man',cosine_sim2,10)
cbResult

1
[(1, 1.0000000000000002), (3, 1.0000000000000002), (5, 0.5773502691896258), (0, 0.43301270189221935), (2, 0.43301270189221935), (4, 0.36514837167011077)]
[(3, 1.0000000000000002), (5, 0.5773502691896258), (0, 0.43301270189221935), (2, 0.43301270189221935), (4, 0.36514837167011077)]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,id,genres,keywords,title,director,cast,sim_score
3,102382,"[action, adventure, fantasy]","[obsession, marvelcomic, sequel]",The Amazing Spider-Man 2,marcwebb,"[andrewgarfield, emmastone, jamiefoxx]","(3, 1.0000000000000002)"
5,76757,"[sciencefiction, fantasy, action]","[jupiter, space, womandirector]",Jupiter Ascending,lillywachowski,"[milakunis, channingtatum, seanbean]","(5, 0.5773502691896258)"
0,559,"[fantasy, action, adventure]","[dualidentity, amnesia, sandstorm]",Spider-Man 3,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]","(0, 0.43301270189221935)"
2,558,"[action, adventure, fantasy]","[dualidentity, loveofone'slife, pizzaboy]",Spider-Man 2,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]","(2, 0.43301270189221935)"
4,557,"[fantasy, action]","[lossoflover, spider, thanksgiving]",Spider-Man,samraimi,"[tobeymaguire, willemdafoe, kirstendunst]","(4, 0.36514837167011077)"


In [133]:
cbData

,id,genres,keywords,title,director,cast
0,559,"[fantasy, action, adventure]","[dualidentity, amnesia, sandstorm]",Spider-Man 3,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
1,1930,"[action, adventure, fantasy]","[lossoffather, vigilante, serum]",The Amazing Spider-Man,marcwebb,"[andrewgarfield, emmastone, rhysifans]"
2,558,"[action, adventure, fantasy]","[dualidentity, loveofone'slife, pizzaboy]",Spider-Man 2,samraimi,"[tobeymaguire, kirstendunst, jamesfranco]"
3,102382,"[action, adventure, fantasy]","[obsession, marvelcomic, sequel]",The Amazing Spider-Man 2,marcwebb,"[andrewgarfield, emmastone, jamiefoxx]"
4,557,"[fantasy, action]","[lossoflover, spider, thanksgiving]",Spider-Man,samraimi,"[tobeymaguire, willemdafoe, kirstendunst]"
5,76757,"[sciencefiction, fantasy, action]","[jupiter, space, womandirector]",Jupiter Ascending,lillywachowski,"[milakunis, channingtatum, seanbean]"
